---
# Buscando registros por termo chave - Sistema Comando

In [4]:
comando = pd.read_csv(data.path + 'comando.csv') 

c:\Users\luisr\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4,10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
comando.head(3)

,EVENTO_ID,EVENTO_TITULO,EVENTO_DESCRICAO,EVENTO_GRAVIDADE,EVENTO_BAIRRO,STATUS,EVENTO_INICIO,EVENTO_INICIO_HORA,EVENTO_FIM,EVENTO_FIM_HORA,EVENTO_PRAZO,EVENTO_LATITUDE,EVENTO_LONGITUDE,POP_TITULO,POP_DESCRICAO,ORGAO_SIGLA,ORGAO_NOME,ACAO
0,2,"Ac mt e tx, na LVermelha, sent Centro, altura ...","Ac mt e tx, na LVermelha, sent Centro, altura ...",BAIXO,NaN,FECHADO,2015-02-23,12:13:00,2015-02-23,14:18:00,NaN,0.0,0.0,Acidente com vítima(s),Acidente com vítima(s),CET-RIO,Companhia de Engenharia de Tráfego,Desfazer o acidente
1,2,"Ac mt e tx, na LVermelha, sent Centro, altura ...","Ac mt e tx, na LVermelha, sent Centro, altura ...",BAIXO,NaN,FECHADO,2015-02-23,12:13:00,2015-02-23,14:18:00,NaN,NaN,0.0,Acidente com vítima(s),Acidente com vítima(s),CET-RIO,Companhia de Engenharia de Tráfego,Organizar o trânsito
2,2,"Ac mt e tx, na LVermelha, sent Centro, altura ...","Ac mt e tx, na LVermelha, sent Centro, altura ...",BAIXO,NaN,FECHADO,2015-02-23,12:13:00,2015-02-23,14:18:00,NaN,0.0,0.0,Acidente com vítima(s),Acidente com vítima(s),BOMBEIROS,Corpo de Bombeiros,Atender vítima(s)


### Função de busca de termos nos registros

In [ ]:
def replace_accents(string):
    item = string.lower()
    for a, b in zip(['á', 'é', 'í', 'ó', 'ú', 'ç'], ['a', 'e', 'i', 'o', 'u', 'c']):
        item = item.replace(a, b)
    return item

def search_records(records, queries, avoid=[]):
    matches = []
    for i, index in enumerate(records.index):
        if i%10000 == 0: print(f'{i+1}/{records.shape[0]}'); co(wait=True)
        record = records.loc[index]
        if type(record)==str:
            record = replace_accents(record)
            query_matches, match_words = [], []
            for query in queries:
                if query in record:
                    words = record.split(' ')
                    for word in words:
                        if query in word:
                            if word not in avoid:
                                match_words.append(word)
                    if match_words:
                        query_matches.append(query)
            if query_matches:
                matches.append([index, query_matches, match_words])
    return pd.DataFrame(matches, columns=['index', 'matched_queries', 'matched_words'])

### Buscando alagamentos, bolsões e lâminas d'água

In [ ]:
queries = ['bolsao', 'bolsoes', 'alagamento', 'lamina', 'enchente', 'inundacão', 'vazamento', 'bueiro']
result_title = search_records(comando['EVENTO_TITULO'].iloc[:], queries, avoid=[])
result_desc = search_records(comando['EVENTO_DESCRICAO'].iloc[:], queries, avoid=[])
result_pop_title = search_records(comando['POP_TITULO'].iloc[:], queries, avoid=[])
result_pop_desc = search_records(comando['POP_DESCRICAO'].iloc[:], queries, avoid=[])

### Análise do resultado

In [86]:
results = [result_title, result_desc, result_pop_title, result_pop_desc]
matches = [comando.loc[result['index']] for result in results]

#### Registros encontrados

In [89]:
list(result.shape for result in results)

[(3658, 3), (7, 3), (3840, 3), (3840, 3)]

#### Ocorrências encontradas

In [72]:
matches['EVENTO_ID'].value_counts().shape

(465,)

#### Análise das informações indisponíveis

In [73]:
matches.isna().sum().sort_values(ascending=False)

EVENTO_ID               0
EVENTO_TITULO           0
EVENTO_DESCRICAO        0
EVENTO_GRAVIDADE        0
EVENTO_BAIRRO         429
STATUS                  0
EVENTO_INICIO           0
EVENTO_INICIO_HORA      0
EVENTO_FIM             10
EVENTO_FIM_HORA        10
EVENTO_PRAZO          501
EVENTO_LATITUDE         0
EVENTO_LONGITUDE        0
POP_TITULO              0
POP_DESCRICAO           0
ORGAO_SIGLA             0
ORGAO_NOME              0
ACAO                    0
dtype: int64